In [1]:
import os
import pandas as pd 
import numpy as np
import re

In [2]:
train = pd.read_csv('./数据集/科创企业成长潜力模型/label/train_2.csv')
train.rename(columns={'label':'Y'}, inplace = True)
test = pd.read_csv('./数据集/科创企业成长潜力模型/upload.csv')

data_all = pd.concat((train[['ID','Y']],test[['ID','Y']]))

In [3]:
folder_path = './数据集/科创企业成长潜力模型/data'  # 替换为你的文件夹路径
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx') and '市场认可' in file]

In [4]:
excel_files

['市场认可_供应商标签数据_上市公司供应商信息.xlsx',
 '市场认可_供应商标签数据_央企供应商信息.xlsx',
 '市场认可_供应商标签数据_政府供应商信息.xlsx']

In [5]:
def extract_nums(str_with_num):
    """
    从一段文本里分离出数字
    :param str_with_num:含数字的文本
    :return: 文本内包含的所有数字列表
    """
    global nums
    try:
        import re
        pattern = re.compile(r'-?\d+\.*\d*')  # 正则匹配数字(包括正负及浮点数)
        nums = re.findall(pattern, str_with_num)
        return float(nums[0])
    except:
        return None

In [6]:
df1 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[0],index_col=0)
df1 = df1.drop_duplicates()
def get_year(x):
    return x[:4]
df1['公告发布时间'] = df1['公告发布时间'].apply(get_year)
def clean_money(x):
    return extract_nums(x)
df1['供应商中标价格'] = df1['供应商中标价格'].apply(clean_money)
df1 = df1.groupby(['企业名称','公告发布时间']).agg({'项目名称':'count', '供应商中标价格': 'sum'}).reset_index()
df1.rename(columns={'企业名称':'ID','项目名称':'上市公司供应商信息_项目名称','供应商中标价格':'上市公司供应商信息_供应商中标价格'}, inplace = True)

features = df1.groupby(['ID']).agg({'公告发布时间':'count'}).reset_index()
for year in range(2018,2024):
    sub_data =df1[df1['公告发布时间']==year].groupby('ID').agg({'上市公司供应商信息_项目名称':'sum', '上市公司供应商信息_供应商中标价格': 'sum'}).reset_index().fillna(0)
    sub_data.rename(columns= {'上市公司供应商信息_项目名称':f'{year}年_上市公司供应商信息_项目名称','上市公司供应商信息_供应商中标价格':f'{year}年_上市公司供应商信息_供应商中标价格'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
data_all = data_all.merge(features,on='ID',how='left')

In [7]:
def get_num(x):
    if isinstance(x, str):
        numbers = re.findall(r'\d+(?:\.\d+)?', x)
        if numbers:
            extracted_number = float(numbers[0])
            return extracted_number
        else:
            return None
    else:
        return None
    
df2 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[1],index_col=0)
df2 = df2.drop_duplicates()
df2['中标价格'] = df2['中标价格'].apply(get_num)
df2['公告发布时间'] = df2['公告发布时间'].map(lambda x: x.year)
df2 = df2.groupby(['供应商公司现用名','公告发布时间']).agg({'项目名称':'count', '中标价格': 'sum'}).reset_index()
df2.rename(columns={'供应商公司现用名':'ID','项目名称':'央企供应商信息_项目名称','中标价格':'央企供应商信息_中标价格'}, inplace = True)

features = df2.groupby(['ID']).agg({'公告发布时间':'count'}).reset_index()
for year in range(2018,2024):
    sub_data =df2[df2['公告发布时间']==year].groupby('ID').agg({'央企供应商信息_项目名称':'sum', '央企供应商信息_中标价格': 'sum'}).reset_index().fillna(0)
    sub_data.rename(columns= {'央企供应商信息_项目名称':f'{year}年_央企供应商信息_项目名称','央企供应商信息_中标价格':f'{year}年_央企供应商信息_中标价格'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
data_all = data_all.merge(features,on='ID',how='left')

In [8]:
df3 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[2],index_col=0)
df3 = df3.drop_duplicates()
df3['供应商中标价格'] = df3['供应商中标价格'].apply(get_num)
df3['公告发布时间'] = pd.to_datetime(df3['公告发布时间']).dt.year
df3 = df3.groupby(['供应商名称','公告发布时间']).agg({'项目名称':'count', '供应商中标价格': 'sum'}).reset_index()
df3.rename(columns={'供应商名称':'ID','项目名称':'政府供应商信息_项目名称','供应商中标价格':'政府供应商信息_中标价格'}, inplace = True)
features = df3.groupby(['ID']).agg({'公告发布时间':'count'}).reset_index()
for year in range(2018,2024):
    sub_data =df3[df3['公告发布时间']==year].groupby('ID').agg({'政府供应商信息_项目名称':'sum', '政府供应商信息_中标价格': 'sum'}).reset_index().fillna(0)
    sub_data.rename(columns= {'政府供应商信息_项目名称':f'{year}年_政府供应商信息_项目名称','政府供应商信息_中标价格':f'{year}年_政府供应商信息_中标价格'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
data_all = data_all.merge(features,on='ID',how='left')

In [9]:
data_all.to_csv('市场认可.csv',index=0)

In [10]:
data_all

,ID,Y,公告发布时间_x,2018年_上市公司供应商信息_项目名称,2018年_上市公司供应商信息_供应商中标价格,2019年_上市公司供应商信息_项目名称,2019年_上市公司供应商信息_供应商中标价格,2020年_上市公司供应商信息_项目名称,2020年_上市公司供应商信息_供应商中标价格,2021年_上市公司供应商信息_项目名称,...,2019年_政府供应商信息_项目名称,2019年_政府供应商信息_中标价格,2020年_政府供应商信息_项目名称,2020年_政府供应商信息_中标价格,2021年_政府供应商信息_项目名称,2021年_政府供应商信息_中标价格,2022年_政府供应商信息_项目名称,2022年_政府供应商信息_中标价格,2023年_政府供应商信息_项目名称,2023年_政府供应商信息_中标价格
0,corp_1001749,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,corp_1001231,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,corp_1004522,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,corp_1001738,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,corp_1003948,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5017,corp_1000552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5018,corp_1001079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5019,corp_1004730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5020,corp_1000350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
